### How Are We Doing?

In the last notebook, you created a working version of SVD for situations even when there are tons of missing values.  This is awesome!  The question now is how well does this solution work?

In this notebook, we are going to simulate exactly what we would do in the real world to tune our recommender.  

Run the cell below to read in the data and get started.

In [92]:
import numpy as np
import pandas as pd

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

1. Using the **reviews** dataframe, perform the following tasks to create a training and validation set of data we can use to test the performance of your SVD algorithm using **off-line** validation techniques.

 * Order the reviews dataframe from earliest to most recent 
 * Pull the first 10000 reviews from  the dataset
 * Make the first 8000/10000 reviews the training data 
 * Make the last 2000/10000 the test data
 * Return the training and test datasets

In [96]:
def create_train_test(reviews, order_by, training_size, testing_size):
    '''    
    INPUT:
    reviews - (pandas df) dataframe to split into train and test
    order_by - (string) column name to sort by
    training_size - (int) number of rows in training set
    testing_size - (int) number of columns in the test set
    
    OUTPUT:
    training_df -  (pandas df) dataframe of the training set
    validation_df - (pandas df) dataframe of the test set
    '''
    ordered_reviews = reviews.sort_values(by=order_by)
    
    training_df = ordered_reviews.iloc[:training_size, :]
    validation_df = ordered_reviews.iloc[training_size:training_size+testing_size, :]
    return training_df, validation_df

In [97]:
# Nothing to change in this or the next cell
# Use our function to create training and test datasets
train_df, val_df = create_train_test(reviews, 'date', 8000, 2000)

In [98]:
# Make sure the dataframes we are using are the right shape
assert train_df.shape[0] == 8000, "The number of rows doesn't look right in the training dataset."
assert val_df.shape[0] == 2000, "The number of rows doesn't look right in the validation dataset"
assert str(train_df.tail(1)['date']).split()[1] == '2013-03-15', "The last date in the training dataset doesn't look like what we expected."
assert str(val_df.tail(1)['date']).split()[1] == '2013-03-18', "The last date in the validation dataset doesn't look like what we expected."
print("Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.")

Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.


In the real world, we might have all of the data up to this final date in the training data.  Then we want to see how well we are doing for each of the new ratings, which show up in the test data.

Below is a working example of the function created in the previous example you can use (or you can replace with your own).

`2.`  Fit the function to the training data with the following hyperparameters: 15 latent features, a learning rate of 0.005, and 250 iterations. This will take some time to run, so you may choose fewer latent features, a higher learning rate, or fewer iteratios if you want to speed up the process.  

**Note:** Again, this might be a good time to take a phone call, go for a walk, or just take a little break.  No need to change the code below unless you would like to make changes to reduce the time needed to obtain results.

In [12]:
def FunkSVD(ratings_mat, latent_features=12, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

In [13]:
# Create user-by-item matrix - nothing to do here
train_user_item = train_df[['user_id', 'movie_id', 'rating', 'timestamp']]
train_data_df = train_user_item.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
train_data_np = np.array(train_data_df)

# Fit FunkSVD with the specified hyper parameters to the training data
user_mat, movie_mat = FunkSVD(train_data_np, latent_features=15, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 10.598544
2 		 5.953297
3 		 4.144326
4 		 3.094323
5 		 2.408043
6 		 1.925133
7 		 1.568415
8 		 1.296078
9 		 1.083185
10 		 0.913754
11 		 0.776991
12 		 0.665314
13 		 0.573223
14 		 0.496624
15 		 0.432406
16 		 0.378174
17 		 0.332059
18 		 0.292600
19 		 0.258640
20 		 0.229263
21 		 0.203736
22 		 0.181465
23 		 0.161968
24 		 0.144847
25 		 0.129773
26 		 0.116470
27 		 0.104703
28 		 0.094276
29 		 0.085018
30 		 0.076783
31 		 0.069447
32 		 0.062901
33 		 0.057050
34 		 0.051812
35 		 0.047116
36 		 0.042900
37 		 0.039109
38 		 0.035695
39 		 0.032616
40 		 0.029836
41 		 0.027322
42 		 0.025046
43 		 0.022982
44 		 0.021109
45 		 0.019406
46 		 0.017857
47 		 0.016446
48 		 0.015159
49 		 0.013985
50 		 0.012911
51 		 0.011929
52 		 0.011030
53 		 0.010206
54 		 0.009450
55 		 0.008757
56 		 0.008119
57 		 0.007533
58 		 0.006993
59 		 0.006496
60 		 0.006038
61 		 0.005616
62 		 0.005226
63 		 0.004865
64 		 

Now that you have created the **user_mat** and **movie_mat**, we can use this to make predictions for how users would rate movies, by just computing the dot product of the row associated with a user and the column associated with the movie.

`3.` Use the comments in the function below to complete the **predict_rating** function.

In [38]:
user_mat[1,:]

array([ 0.50012684,  0.80406459,  0.61767635,  0.33846321,  0.64656298,
        0.8049172 ,  0.94233914,  1.26710714,  0.9282181 ,  0.6250358 ,
        0.51311048,  0.51160688,  1.1630962 ,  0.47388083,  1.19085143])

In [99]:
def predict_rating(user_matrix, movie_matrix, user_id, movie_id):
    '''
    INPUT:
    user_matrix - user by latent factor matrix
    movie_matrix - latent factor by movie matrix
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    
    OUTPUT:
    pred - the predicted rating for user_id-movie_id according to FunkSVD
    '''
    # Use the training data to create a series of users and movies that matches the ordering in training data
    row = np.where((train_data_df.index.values) == user_id)[0][0]
    col = np.where((train_data_df.columns.values) == movie_id)[0][0]

    # User row and Movie Column
    u = user_matrix[row, :]
    v = movie_matrix[:, col]
    # Take dot product of that row and column in U and V to make prediction
    pred = np.dot(u, v)
    return pred

In [100]:
# Test your function with the first user-movie in the user-movie matrix (notice this is a nan)
pred_val = predict_rating(user_mat, movie_mat, 8, 2844)
pred_val

7.4782386720763627

It is great that you now have a way to make predictions. However it might be nice to get a little phrase back about the user, movie, and rating.

`4.` Use the comments in the function below to complete the **predict_rating** function.  

**Note:** The movie name doesn't come back in a great format, so you can see in the solution I messed around with it a bit just to make it a little nicer.

In [101]:
def print_prediction_summary(user_id, movie_id, prediction):
    '''
    INPUT:
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    prediction - the predicted rating for user_id-movie_id
    
    OUTPUT:
    None - prints a statement about the user, movie, and prediction made
    
    '''
    print('For user {} movie {}, the predicted rating is {:.0f}'.format(user_id, movie_id, prediction))


In [102]:
# Test your function the the results of the previous function
print_prediction_summary(8, 2844, pred_val)

For user 8 movie 2844, the predicted rating is 7


Now that we have the ability to make predictions, let's see how well our predictions do on the test ratings we already have.  This will give an indication of how well we have captured the latent features, and our ability to use the latent features to make predictions in the future!

`5.` For each of the user-movie rating in the **val_df** dataset, compare the actual rating given to the prediction you would make.  How do your predictions do?  Do you run into any problems?  If yes, what is the problem?  Use the document strings and comments below to assist as you work through these questions.

In [107]:
val_df.head(7)

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
650588,49056,1598822,8,1363308721,2013-03-15 00:52:01,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
650569,49056,289879,9,1363308742,2013-03-15 00:52:22,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
650585,49056,1563738,9,1363308780,2013-03-15 00:53:00,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
650583,49056,1458175,4,1363308799,2013-03-15 00:53:19,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
378686,28599,103639,8,1363309112,2013-03-15 00:58:32,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
667660,50593,1560985,4,1363309202,2013-03-15 01:00:02,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
385306,29000,287978,9,1363309214,2013-03-15 01:00:14,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [104]:
def validation_comparison(val_df, num_preds):
    '''
    INPUT:
    val_df - the validation dataset created in the third cell above
    num_preds - (int) the number of rows (going in order) you would like to make predictions for
    
    OUTPUT:
    Nothing returned - print a statement about the prediciton made for each row of val_df from row 0 to num_preds
    '''
    for i in range(num_preds):
        user_id = val_df['user_id'].iloc[i]
        movie_id = val_df['movie_id'].iloc[i]
        actual_rating = val_df['rating'].iloc[i]
        print('The actual rating is {}'.format(actual_rating))
        pred_val = predict_rating(user_mat, movie_mat, user_id, movie_id)
        print_prediction_summary(user_id, movie_id, pred_val)
    
# Perform the predicted vs. actual for the first 6 rows.  How does it look?
validation_comparison(val_df, 6)        

The actual rating is 8
For user 49056 movie 1598822, the predicted rating is 6
The actual rating is 9
For user 49056 movie 289879, the predicted rating is 7
The actual rating is 9
For user 49056 movie 1563738, the predicted rating is 8
The actual rating is 4
For user 49056 movie 1458175, the predicted rating is 7
The actual rating is 8
For user 28599 movie 103639, the predicted rating is 8
The actual rating is 4
For user 50593 movie 1560985, the predicted rating is 3


In [105]:
1560985 in train_data_df.columns
np.where((train_data_df.columns.values) == 287978)

(array([], dtype=int64),)

In [106]:
# Perform the predicted vs. actual for the first 7 rows.  What happened?
validation_comparison(val_df, 7)        

The actual rating is 8
For user 49056 movie 1598822, the predicted rating is 6
The actual rating is 9
For user 49056 movie 289879, the predicted rating is 7
The actual rating is 9
For user 49056 movie 1563738, the predicted rating is 8
The actual rating is 4
For user 49056 movie 1458175, the predicted rating is 7
The actual rating is 8
For user 28599 movie 103639, the predicted rating is 8
The actual rating is 4
For user 50593 movie 1560985, the predicted rating is 3
The actual rating is 9


IndexError: index 0 is out of bounds for axis 0 with size 0

** A statement about why you think what happened happened.**

In [109]:
287978 in train_data_df.columns

False

In the training data, no user have ever watched movie 287978